In [1]:
import numpy as np
import random
import math
from collections import deque
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.


In [55]:
m = 10 # number of locations, ranges from 1 ..... m
t = 24 # number of hours, ranges from 0 .... t-1
d = 30 # number of days, ranges from 0 ... d-1
episode_length = 1000
fixed_act_Set = 15
epsilon = 0.99
learning_rate = 0.01
batch_size = 50
gamma = 0.95
epsilon_min = 0.01
epsilon_decay = 0.995
Time_matrix = np.random.randint(1, 11,(m, m))
Q_value = np.random.randint(-1, 0,(m*t*d, m*m))
actions = []
for i in range(m):
    for j in range(m):
        actions.append([i,j])
states = []
for i in range(m):
    for j in range(t):
        for k in range(d):
            states.append([i,j,k])

In [3]:
def state_trans(state,action,m,t,d):
    state_encod = np.zeros(m+t+d+m+m)
    state_encod.reshape(1,84)
    state_encod[current_state[0]] = 1
    state_encod[m+current_state[1]] = 1
    state_encod[m+t+current_state[2]] = 1
    state_encod[m+t+d+action[0]] = 1
    state_encod[m+t+d+m+action[1]] = 1
    return state_encod   

In [4]:
def act(state,Q_value,epsilon,actions):
    if np.random.rand() <= epsilon:
        index = np.random.randint(0, m*m,(1, 1))
        if actions[np.int(index)][0] == actions[np.int(index)][1]:
            return act(state,Q_value,epsilon,actions)
        else:
            return index
    else:
        index = np.argmax(Q_value[state])
        if actions[np.int(index)][0] == actions[np.int(index)][1]:
            return act(state,Q_value,epsilon,actions)
        else:
            return index

In [5]:
def reward_func(state, action, time_matrix):
    start_loc, time, day = state
    pickup, drop = action
    if pickup == 0 and drop == 0:
        return -1
    else:
        return time_matrix[pickup, drop] - time_matrix[start_loc, pickup]

In [6]:
def next_state_func(state, action, time_matrix, t, d):
    start_loc, time, day = state
    pickup, drop = action
    if pickup == 0 and drop == 0:
        time_elapsed = 1
        drop = start_loc
    else:
        time_elapsed = time_matrix[start_loc, pickup] + time_matrix[pickup, drop]
    time_next = (time + time_elapsed) % t
    day_next = (day + (time + time_elapsed)//t) % d
    return drop, time_next, day_next

In [53]:
def prediction_DQN(m,t,d,actions,X,index_State, Q_value):
    state_pred = X[0][0:m+t+d]
    X_test = np.zeros((len(actions), t+d+3*m))
    for i in range(len(actions)):
        action_encod_p = np.zeros(m+m)
        action_encod_p[actions[i][0]] = 1
        action_encod_p[m + actions[i][1]] = 1
        dummy = np.hstack((state_pred,action_encod_p))
        X_test[i,:] = dummy
    prediction = model.predict(X_test)
    prediction = prediction.reshape(len(actions))
    return prediction

In [22]:
Q_value[index_State][i] = model.predict(X_test)


array([[7.7269607]], dtype=float32)

In [7]:
# Deep learning model
model = Sequential()
model.add(Dense(100, input_dim = np.int(m+t+d+m+m),activation ='relu'))
model.add(Dense(100,activation ='relu'))
model.add(Dense(1,activation ='linear'))
model.compile(loss='mse',optimizer=Adam(lr=learning_rate))

In [56]:
# code for genrating episode
memory = deque(maxlen=2000)
current_state =  (np.random.randint(0, m),np.random.randint(0, t),np.random.randint(0, d))
count  =  0
for epi_len in range(episode_length):
    # pick a random action
    count = count + 1
    index_State = states.index([current_state[0],current_state[1],current_state[2]])
    action = actions[np.int(act(index_State,Q_value,epsilon,actions))]
    reward = reward_func(current_state, action, Time_matrix)
    next_state = next_state_func(current_state, action, Time_matrix, t, d)
    #print("State: ",current_state, " Action: ", action, " Reward: ", reward, " Nextstate: ", next_state)
    X = state_trans(current_state,action,m,t,d)
    index_Next_State = states.index([next_state[0],next_state[1],next_state[2]])
    memory.append((X,reward,index_Next_State))
    current_state = next_state
    if count > batch_size:
        mini_batch = random.sample(memory, batch_size)
        for X, reward, index_Next_State in mini_batch:
            target = reward + gamma*np.argmax(Q_value[index_Next_State])
            X = X.reshape(1,t+d+3*m)
            target = target.reshape(1,1)
            model.fit(X,target, epochs = 1, verbose = 0)
            Q_value[index_Next_State] = prediction_DQN(m,t,d,actions,X,index_Next_State, Q_value)
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay
    if epi_len % 50 == 0:
        print("Reward:{},Episode:{}, Epsilon:{}".format(reward,epi_len,epsilon))

Reward:0,Episode:0, Epsilon:0.99
Reward:3,Episode:50, Epsilon:0.98505
Reward:-1,Episode:100, Epsilon:0.7666767843404657
Reward:-4,Episode:150, Epsilon:0.5967141684651915
Reward:-4,Episode:200, Epsilon:0.46443013029723174
Reward:-4,Episode:250, Epsilon:0.36147180229136094
Reward:0,Episode:300, Epsilon:0.28133804274959967
Reward:-3,Episode:350, Epsilon:0.21896893145312793
Reward:9,Episode:400, Epsilon:0.17042626895787213
Reward:7,Episode:450, Epsilon:0.13264490518426955
Reward:1,Episode:500, Epsilon:0.10323919533609638
Reward:-9,Episode:550, Epsilon:0.0803523621117462
Reward:0,Episode:600, Epsilon:0.06253925242169864
Reward:5,Episode:650, Epsilon:0.0486750854694935
Reward:2,Episode:700, Epsilon:0.03788443023729618
Reward:3,Episode:750, Epsilon:0.029485927771078575
Reward:1,Episode:800, Epsilon:0.022949267841049455
Reward:-7,Episode:850, Epsilon:0.017861703336220346
Reward:0,Episode:900, Epsilon:0.013901987997215145
Reward:-2,Episode:950, Epsilon:0.010820091826450088


In [51]:
prediction = model.predict(X_test)

